# Data Sampling for Responsible AI Principles

This notebook demonstrates responsible sampling techniques for evaluating Large Language Models (LLMs)
with a focus on bias mitigation and ethical considerations. We'll cover:

1. Why balanced sampling matters for AI fairness
2. Implementation of different sampling techniques
3. Evaluation across diverse demographic groups
4. Analysis of model performance disparities

## Installation
### Prerequisites
Ensure you have Python 3.8+ installed. The project dependencies are listed in requirements.txt:
```bash 
pip install -r requirements.txt
```

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import Dict, List, Tuple, Optional
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import torch
from transformers import pipeline
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


# Set random seed for reproducibility
np.random.seed(42)

In [ ]:
import nltk
nltk.download(['stopwords', 'punkt_tab'])

In [ ]:
class ResponsibleDataSampler:
    """
    A class to handle responsible data sampling and analysis for NLP tasks.
    """
    
    def __init__(self, random_seed: int = 42):
        """
        Initialize the sampler with optional random seed.
        
        Args:
            random_seed: Seed for reproducibility
        """
        np.random.seed(random_seed)
        self.sentiment_analyzer = pipeline(
            "sentiment-analysis",
            model="distilbert-base-uncased-finetuned-sst-2-english",
            top_k=None
        )
        
    def create_synthetic_dataset(
        self,
        n_samples: int = 1000,
        weighted: bool = True
    ) -> pd.DataFrame:
        """
        Creates a synthetic dataset with demographic weights.
        
        Args:
            n_samples: Number of samples to generate
            weighted: Whether to apply demographic weights
            
        Returns:
            pd.DataFrame: Synthetic dataset
        """
        # Define demographics with realistic weights
        demographics = {
            'gender': {
                'categories': ['female', 'male', 'non_binary'],
                'weights': [0.49, 0.48, 0.03] if weighted else None
            },
            'age_group': {
                'categories': ['18-25', '26-40', '41-60', '60+'],
                'weights': [0.2, 0.35, 0.3, 0.15] if weighted else None
            },
            'region': {
                'categories': ['north', 'south', 'east', 'west'],
                'weights': [0.3, 0.35, 0.15, 0.2] if weighted else None
            },
            'language': {
                'categories': ['english', 'spanish', 'mandarin', 'arabic'],
                'weights': [0.45, 0.25, 0.2, 0.1] if weighted else None
            }
        }
        
        # Generate demographic data
        df_dict = {}
        for feature, data in demographics.items():
            df_dict[feature] = np.random.choice(
                data['categories'],
                n_samples,
                p=data['weights'] if weighted else None
            )
            
        # Generate synthetic text with demographic-specific patterns
        texts = []
        for i in range(n_samples):
            sentiment = np.random.choice(['positive', 'negative'])
            if sentiment == 'positive':
                template = np.random.choice([
                    "I really enjoyed this product!",
                    "The service was excellent.",
                    "This helps me a lot.",
                ])
            else:
                template = np.random.choice([
                    "I'm disappointed with this product.",
                    "The service was poor.",
                    "This doesn't work for me.",
                ])
            texts.append(template)
        
        df_dict['text'] = texts
        
        # Create DataFrame
        df = pd.DataFrame(df_dict)
        
        # Add sentiment labels with demographic bias
        df['label'] = np.where(
            (df['age_group'] == '18-25') & (df['gender'] == 'female'),
            np.random.choice(['positive', 'negative'], n_samples, p=[0.7, 0.3]),
            np.random.choice(['positive', 'negative'], n_samples, p=[0.5, 0.5])
        )
        
        return df
    
    def analyze_text_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Analyzes text features across demographic groups.
        
        Args:
            df: Input DataFrame
            
        Returns:
            pd.DataFrame: DataFrame with text analysis features
        """
        stop_words = set(stopwords.words('english'))
        
        def get_text_metrics(text: str) -> Dict[str, float]:
            tokens = word_tokenize(text.lower())
            tokens_no_stop = [t for t in tokens if t not in stop_words]
            
            return {
                'text_length': len(text),
                'word_count': len(tokens),
                'vocabulary_diversity': len(set(tokens_no_stop)) / len(tokens_no_stop) if tokens_no_stop else 0
            }
        
        # Calculate text metrics
        text_metrics = df['text'].apply(get_text_metrics).apply(pd.Series)
        df = pd.concat([df, text_metrics], axis=1)
        
        return df
    
    def perform_stratified_sampling(
        self,
        df: pd.DataFrame,
        strat_cols: List[str],
        sample_size: int
    ) -> pd.DataFrame:
        """
        Performs stratified sampling across specified columns.
        
        Args:
            df: Input DataFrame
            strat_cols: Columns to stratify on
            sample_size: Desired sample size
            
        Returns:
            pd.DataFrame: Stratified sample
        """
        df = df.copy()
        df['strata'] = df[strat_cols].apply(lambda x: '_'.join(x.astype(str)), axis=1)
        
        # Calculate sample sizes for each stratum
        stratum_sizes = (df['strata'].value_counts(normalize=True) * sample_size).round().astype(int)
        
        # Sample from each stratum
        samples = []
        for stratum, size in stratum_sizes.items():
            stratum_df = df[df['strata'] == stratum]
            if len(stratum_df) < size:
                samples.append(stratum_df)  # Take all if not enough samples
            else:
                samples.append(stratum_df.sample(n=size))
        
        return pd.concat(samples).drop('strata', axis=1)
    
    def analyze_model_biases(
        self,
        df: pd.DataFrame,
        demographic_cols: List[str]
    ) -> Dict:
        """
        Analyzes model predictions across demographic groups.
    
        Args:
            df: DataFrame with text data
            demographic_cols: Demographic columns to analyze
        
        Returns:
            Dict: Bias analysis results
        """
        bias_metrics = {}
    
        for col in demographic_cols:
            bias_metrics[col] = {}
        
            for group in df[col].unique():
                group_texts = df[df[col] == group]['text'].tolist()
            
                # Get model predictions
                predictions = self.sentiment_analyzer(group_texts)
            
                # Handle different output formats from the pipeline
                if isinstance(predictions, list) and isinstance(predictions[0], list):
                    # When top_k=None, format is list of lists
                    pred_labels = [p[0]['label'] for p in predictions]
                    confidences = [p[0]['score'] for p in predictions]
                else:
                    # Standard format when top_k is not specified
                    pred_labels = [p['label'] for p in predictions]
                    confidences = [p['score'] for p in predictions]
            
                # Calculate metrics
                bias_metrics[col][group] = {
                    'representation': len(group_texts) / len(df),
                    'avg_confidence': np.mean(confidences),
                    'pred_distribution': Counter(pred_labels)
                }
    
        return bias_metrics
    
    def plot_demographic_distributions(
        self,
        df: pd.DataFrame,
        demographic_cols: List[str]
    ) -> None:
        """
        Plots the distribution of samples across demographic groups.
        
        Args:
            df: Input DataFrame
            demographic_cols: Demographic columns to plot
        """
        n_cols = len(demographic_cols)
        fig, axes = plt.subplots(1, n_cols, figsize=(5*n_cols, 4))
        
        for i, col in enumerate(demographic_cols):
            sns.countplot(data=df, x=col, ax=axes[i])
            axes[i].set_title(f'{col} Distribution')
            axes[i].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()
    
    def plot_bias_metrics(self, bias_metrics: Dict) -> None:
        """
        Visualizes bias metrics across demographic groups.
        
        Args:
            bias_metrics: Dictionary of bias metrics
        """
        n_metrics = len(bias_metrics)
        fig, axes = plt.subplots(1, n_metrics, figsize=(6*n_metrics, 4))
        
        for i, (demographic, group_metrics) in enumerate(bias_metrics.items()):
            groups = list(group_metrics.keys())
            confidences = [m['avg_confidence'] for m in group_metrics.values()]
            
            sns.barplot(x=groups, y=confidences, ax=axes[i])
            axes[i].set_title(f'{demographic} - Avg Confidence')
            axes[i].set_ylim(0, 1)
            axes[i].tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

# Code Implementation

In [ ]:
# Initialize sampler
sampler = ResponsibleDataSampler()
    
# Create and analyze dataset
df = sampler.create_synthetic_dataset(n_samples=500, weighted=True)
df = sampler.analyze_text_features(df)

In [ ]:
# Perform stratified sampling
demographic_cols = ['gender', 'age_group', 'region', 'language']
sample_size = 500
stratified_sample = sampler.perform_stratified_sampling(
    df,
    demographic_cols,
    sample_size
)

In [ ]:
# Analyze biases
bias_metrics = sampler.analyze_model_biases(
    stratified_sample,
    demographic_cols
)

In [ ]:
# Plot results
print("\n=== Dataset Overview ===")
print(df.head())
    
print("\n=== Distribution of Demographics ===")
sampler.plot_demographic_distributions(df, demographic_cols)

In [ ]:
print("\n=== Bias Analysis ===")
sampler.plot_bias_metrics(bias_metrics)

# Things to Remember 


1. Representative Sampling
   - Ensures all demographic groups are adequately represented
   - Helps identify and address performance disparities
   - Critical for building inclusive AI systems

2. Stratified Sampling Benefits
   - Maintains population proportions
   - Reduces sampling bias
   - Enables meaningful comparison across groups

3. Fairness Evaluation
   - Regular monitoring of performance across groups
   - Identification of potential biases
   - Guide for model improvements

4. Ethical Considerations
   - Privacy protection in demographic data collection
   - Transparent reporting of limitations
   - Regular updates to sampling strategies

5. Best Practices
   - Document sampling methodology
   - Version control for reproducibility
   - Regular audits of sampling process
   - Consultation with domain experts